In [2]:
import pandas as pd
import geopandas as gpd
import requests, json

Let's get the charger location data again:

In [126]:
apiKey = "eCN7llpPT79TmygqmvC71QdnnWdOquoRdnCR1DXo"
nrelString = "https://developer.nrel.gov/api/alt-fuel-stations/v1.geojson?api_key={}&fuel_type=ELEC&state=CA".format(apiKey)
chargers = gpd.read_file(nrelString)

In [127]:
chargers = chargers[chargers['access_code'] == 'public']

In [135]:
chargers['year'] = chargers['open_date'].str.slice(start = 0, stop = 4)


In [136]:
chargers.head()

,access_code,access_days_time,access_detail_code,cards_accepted,date_last_confirmed,expected_date,fuel_type_code,groups_with_access_code,id,open_date,...,ng_vehicle_class,access_days_time_fr,intersection_directions_fr,bd_blends_fr,groups_with_access_code_fr,ev_pricing_fr,ev_network_ids,federal_agency,geometry,year
2,public,24 hours daily; pay lot,None,None,2020-11-09,None,ELEC,Public,1523,1995-08-30,...,None,None,None,None,Public,None,None,None,POINT (-118.27139 34.04054),1995
10,public,24 hours daily,None,None,2021-10-12,None,ELEC,Public,1583,1996-10-15,...,None,None,None,None,Public,None,None,None,POINT (-118.06400 34.06872),1996
12,public,24 hours daily,None,None,2021-07-14,None,ELEC,Public,6355,1997-07-30,...,None,None,None,None,Public,None,None,None,POINT (-117.24300 32.89947),1997
13,public,Dealership business hours,CALL,None,2021-12-09,None,ELEC,Public - Call ahead,6405,2012-12-11,...,None,None,None,None,Public - Appeler à l'avance,None,None,None,POINT (-118.46837 34.22167),2012
14,public,6am-12am daily,None,None,2020-02-06,None,ELEC,Public,6425,1997-08-30,...,None,None,None,None,Public,None,None,None,POINT (-117.45905 33.90991),1997


In [137]:
chargers.to_pickle("data/chargers")

Now let's get EV sale data (this is on zip-code level):

In [114]:
sales = pd.read_csv("data/ZEV_Sales_by_Zipcode.csv")
sales = sales[sales['Fuel Type'] == 'Electric']

In [6]:
zipSales = sales.groupby("ZIP").sum()

There are a couple zipcodes that aren't in CA that we need to drop: one is a random one with only one sale, and one (99999) seems to be used when there was no zipcode associated with the sale, so it's much larger than the other values. 
http://www.structnet.com/instructions/zip_min_max_by_state.html

In [7]:
zipSales = zipSales.drop(89061)
zipSales = zipSales.drop(99999)

In [8]:
#also, let's get rid of data year, bc it's not meaningful to us anymore; we just want total # of EV's sold
zipSales = zipSales.drop(columns = "Data Year")

In [9]:
zipSales['Number of Vehicles'].describe()

count    2162.000000
mean      519.116559
std       901.209629
min         1.000000
25%         6.000000
50%        67.000000
75%       706.500000
max      8123.000000
Name: Number of Vehicles, dtype: float64

Okay, now let's aggregate the charger locations by zipcode. We need to read in the zipcode boundaries:

In [10]:
zipcodes = gpd.read_file("data/Zipcode Boundaries/cb_2019_us_zcta510_500k.shp")

Now let's spatial join to count the number of chargers within each zip code:

In [11]:
joined_chargers = gpd.sjoin(chargers, zipcodes.to_crs("EPSG:4326"), predicate = "within", how = "left")

In [12]:
joined_chargers.head()

,access_code,access_days_time,access_detail_code,cards_accepted,date_last_confirmed,expected_date,fuel_type_code,groups_with_access_code,id,open_date,...,ev_pricing_fr,ev_network_ids,federal_agency,geometry,index_right,ZCTA5CE10,AFFGEOID10,GEOID10,ALAND10,AWATER10
2,public,24 hours daily; pay lot,None,None,2020-11-09,None,ELEC,Public,1523,1995-08-30,...,None,None,None,POINT (-118.27139 34.04054),26488.0,90015,8600000US90015,90015,4430399.0,0.0
10,public,24 hours daily,None,None,2021-10-12,None,ELEC,Public,1583,1996-10-15,...,None,None,None,POINT (-118.06400 34.06872),8696.0,91731,8600000US91731,91731,9881410.0,169467.0
12,public,24 hours daily,None,None,2021-07-14,None,ELEC,Public,6355,1997-07-30,...,None,None,None,POINT (-117.24300 32.89947),30854.0,92037,8600000US92037,92037,33842473.0,5226832.0
13,public,Dealership business hours,CALL,None,2021-12-09,None,ELEC,Public - Call ahead,6405,2012-12-11,...,None,None,None,POINT (-118.46837 34.22167),16093.0,91343,8600000US91343,91343,15310106.0,34625.0
14,public,6am-12am daily,None,None,2020-02-06,None,ELEC,Public,6425,1997-08-30,...,None,None,None,POINT (-117.45905 33.90991),31592.0,92503,8600000US92503,92503,77488367.0,10935445.0


In [13]:
group_chargers = joined_chargers.groupby("ZCTA5CE10").access_code.count()

In [14]:
group_chargers

ZCTA5CE10
00987     1
21032     1
21043     1
90001     1
90002     6
         ..
96146     2
96148     2
96150    22
96155     1
96161    23
Name: access_code, Length: 1129, dtype: int64

Again, there are some outside CA, so let's drop them:

In [15]:
group_chargers = group_chargers.drop('00987')

In [16]:
group_chargers = group_chargers.drop('21032')
group_chargers = group_chargers.drop('21043')

Okay, so now we should have the number of sales per zip code as well as the number of chargers.

In [17]:
zipSales

,Number of Vehicles
ZIP,
90001,168
90002,126
90003,124
90004,1576
90005,687
...,...
96151,2
96158,4
96160,8


In [18]:
group_chargers

ZCTA5CE10
90001     1
90002     6
90003     5
90004     3
90005     5
         ..
96146     2
96148     2
96150    22
96155     1
96161    23
Name: access_code, Length: 1126, dtype: int64

Some zipcodes have zero chargers, which means that they don't show up in the series, but we want them to, right? Anyways, we can join the two dataframes together to start plotting/comparing

In [19]:
joined_zipcodes = zipSales.join(group_chargers)
joined_zipcodes.head()

,Number of Vehicles,access_code
ZIP,,
90001,168,NaN
90002,126,NaN
90003,124,NaN
90004,1576,NaN
90005,687,NaN


In [20]:
#fix join
group_chargers.index = group_chargers.index.astype(int)
joined_zipcodes = zipSales.join(group_chargers)
joined_zipcodes.head(20)

,Number of Vehicles,access_code
ZIP,,
90001,168,1.0
90002,126,6.0
90003,124,5.0
90004,1576,3.0
90005,687,5.0
90006,435,4.0
90007,248,66.0
90008,645,5.0
90009,6,NaN


In [21]:
joined_zipcodes.rename(columns = {'access_code':'chargers'}, inplace = True)

In [22]:
#set nan to zero
joined_zipcodes.chargers = joined_zipcodes.chargers.fillna(0)

In [23]:
joined_zipcodes.head(10)

,Number of Vehicles,chargers
ZIP,,
90001,168,1.0
90002,126,6.0
90003,124,5.0
90004,1576,3.0
90005,687,5.0
90006,435,4.0
90007,248,66.0
90008,645,5.0
90009,6,0.0


In [24]:
zipcodes['ZCTA5CE10'] = zipcodes['ZCTA5CE10'].astype(int)

In [25]:
zipcode_gdf = gpd.GeoDataFrame(joined_zipcodes, geometry = zipcodes.set_index("ZCTA5CE10").geometry)

Okay, now we can look at the number of chargers/ev to get an idea of where there's a mismatch!

In [80]:
#multiply by 1000 just for rounder numbers
zipcode_gdf['Chargers/EV'] = 1000*(zipcode_gdf['chargers']/zipcode_gdf['Number of Vehicles'])
zipcode_gdf.head()

,Number of Vehicles,chargers,geometry,Chargers/EV,EVs/Charger
ZIP,,,,,
90001,168,1.0,"POLYGON ((-118.26519 33.98923, -118.25644 33.9...",5.952381,168.000000
90002,126,6.0,"POLYGON ((-118.26512 33.96013, -118.25858 33.9...",47.619048,21.000000
90003,124,5.0,"POLYGON ((-118.28320 33.98914, -118.28039 33.9...",40.322581,24.800000
90004,1576,3.0,"POLYGON ((-118.33858 34.08346, -118.33096 34.0...",1.903553,525.333333
90005,687,5.0,"MULTIPOLYGON (((-118.29291 34.06360, -118.2916...",7.278020,137.400000


In [112]:
#okay, now let's save the df to a pickle that we can use in the analysis notebook
zipcode_gdf.to_pickle("data/q2")